In [2]:
root_path = '/home/chris/Documents/UT/SWTest/project/QualitySoftware'

In [3]:
class_filter = ""
llm_filter = ""
prompt_filter = ""

In [ ]:
import os
import yaml
import subprocess
import sys

In [5]:
main_classes_path = os.path.join(root_path, 'src', 'main', 'java')
test_classes_path = os.path.join(root_path, 'src', 'test', 'java')

In [10]:
packages = []
for root, _, files in os.walk(main_classes_path):
    print("Checking path: ", root)
    if (files):
        package = root.split('/')[-1]
        classes = []
        for file in files:
            if file.endswith('.java'):
                classes.append(file.split('.java')[0])
        packages.append({package: classes})

print(yaml.dump(packages, sort_keys=True))

Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/anagrams
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/binaryTree
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/orangesRotting
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/numIslands
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/cloneGraph
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/NQueens
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/MedianSortedArrays
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/RegularExpressionMatching
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/java/trityp
- anagrams:
  - Anagram

In [9]:
packages = []
for root, _, files in os.walk(test_classes_path):
    print("Checking path: ", root)
    if (files):
        package = ".".join(root.split('/')[-3:])
        classes = []
        for file in files:
            print(file)
            if file.endswith('.java'):
                classes.append(file.split('.java')[0])
        packages.append({package: classes})

print(yaml.dump(packages, sort_keys=True))

Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/metaAI
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/metaAI/prompt0
AnagramsTest.java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/ai21Jamba
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/ai21Jamba/prompt1
anagramsai21JambaTest.java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/ai21Jamba/prompt3
anagramsai21JambaTest.java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/test/java/anagrams/ai21Jamba/prompt2
anagramsai21JambaTest.java
Checking path:  /home/chris/Documents/UT/SWTest/project/QualityS

In [14]:
with open('../classes.yml', 'r') as f:
    classes = yaml.safe_load(f)

print(type(classes))

for classy in classes.keys():
    package = classy.get('package')
    if (classy_package := packages.get('package')):
        if (classy_package == classy):
            print(f"Validated {classy}")
        else:
            print(f"Error, something doesn't match {classy_package}, {classy}")
    else:
        print(f"Error, package {package} not found")
        continue

<class 'dict'>


AttributeError: 'str' object has no attribute 'get'

In [167]:
class Project:
    def __init__(self):
        self.name = "Quality Software"
        self.packages: list[Package] = []

    def get_package(self, package: str) -> Package:
        for p in self.packages:
            if (p.name == package):
                return p
        return None
        
    def get_classes(self, package_filter=None) -> list[Class]:
        classes: list[Class] = []
        for package in self.packages:
            if (package_filter is None or package.name == package_filter):
                for c in package.classes:
                    classes.append(c)
        return classes
    
    def get_test_classes(self, package_filter:str=None, llm_filter:str=None, prompt_filter:str=None) -> list[TestClass]:
        test_classes: list[TestClass] = []
        for package in self.packages:
            if (package_filter is None or package.name == package_filter):
                for test_class in package.test_classes:
                    if ((llm_filter is None or test_class.llm == llm_filter)
                        and (prompt_filter is None or test_class.prompt == prompt_filter)):
                        test_classes.append(test_class)
        return test_classes

class Class:
    def __init__(self, name, package):
        self.name = name
        self.package = package

    def get_class_path(self):
        """Return the full class path in the format package.class_name"""
        return f"{self.package}.{self.name}"

    def get_directory_path(self):
        """Return the relative directory path in the format src/main/java/package/classname"""
        return os.path.join("src", "main", "java", *self.package.split('.'), f"{self.name}.java")

    def __repr__(self):
        return f"Class(name={self.name}, package={self.package})"


class TestClass:
    def __init__(self, name, package, llm, prompt):
        self.name = name
        self.package = package
        self.llm = llm
        self.prompt = prompt

    def get_class_path(self):
        """Return the full test class path in the format package.llm.prompt.test_class_name"""
        return f"{self.package}.{self.llm}.{self.prompt}.{self.name}"

    def get_directory_path(self):
        """Return the relative directory path in the format src/test/java/package/llm/prompt/testclassname"""
        return os.path.join("src", "test", "java", *self.package.split('.'), self.llm, self.prompt, f"{self.name}.java")

    def __repr__(self):
        return f"TestClass(name={self.name}, package={self.package}, llm={self.llm}, prompt={self.prompt})"

class Package:
    def __init__(self, name):
        self.name = name
        self.classes = []
        self.test_classes = []

    def add_class(self, class_obj):
        """Add a Class object to the package"""
        self.classes.append(class_obj)

    def add_test_class(self, test_class_obj):
        """Add a TestClass object to the package"""
        self.test_classes.append(test_class_obj)

    def get_class(self, class_name: str) -> Class:
        """Retrieve a Class by name"""
        for c in self.classes:
            if (c.name == class_name):
                return c
        return None

    def get_test_class(self, test_class_name: str) -> TestClass:
        """Retrieve a TestClass by name"""
        for tc in self.test_classes:
            if (tc.name == test_class_name):
                return tc
        return None

    def yaml_dump(self) -> str:
        """Return a YAML-like representation of the package structure"""
        package_data = {
            'package': self.name,
            'classes': [cls.get_class_path() for cls in self.classes],
            'test_classes': [
                {
                    'class_path': test_cls.get_class_path(),
                    'llm': test_cls.llm,
                    'prompt': test_cls.prompt
                } for test_cls in self.test_classes
            ]
        }
        return yaml.dump(package_data, sort_keys=False, indent=4)

    def __repr__(self):
        return f"Package(name={self.name}, classes={self.classes}, test_classes={self.test_classes})"


In [168]:
root_path = ('../')
class_path = root_path + 'src/main/java/'
testclass_path = root_path + 'src/test/java/'
filter_empty_test_classes = True
debug = False


In [169]:
def isEmptyTestClass(tc: TestClass) -> bool:
    dummy_string = "This is a placeholder test class"
    with open('../' + tc.get_directory_path(), 'r') as f:
        if dummy_string in f.read():
            return True
    return False

In [170]:
def debug_print(*args, **kwargs):
    if debug:
        print("DEBUG:", end="")
        print(*args, **kwargs)


In [171]:
packages = os.listdir(class_path)
my_packages: list[Package] = []
for p in packages:
    debug_print(f"Looking for classes under package {p}")
    package = Package(p)
    classes = os.listdir(class_path + p)
    for c in classes:
        if c.endswith('.java'):
            cc = Class(name=c.split('.java')[0], package=p)
            debug_print(f"\tFound class {cc.get_class_path()}")
            package.add_class(cc)
    llms = os.listdir(testclass_path + p)
    for llm in llms:
        prompts = os.listdir(f"{testclass_path}{p}/{llm}/")
        for prompt in prompts:
            testclasses = os.listdir(f'{testclass_path}{p}/{llm}/{prompt}/')
            for t in testclasses:
                if t.endswith('.java'):
                    tc = TestClass(
                        name=t.split('.java')[0],
                        package=p,
                        llm=llm,
                        prompt=prompt
                    )
                    if (not (filter_empty_test_classes and isEmptyTestClass(tc))):                    
                        debug_print(f"\t\tFound test class {tc.get_class_path()}")
                        package.add_test_class(tc)
                    else:
                        debug_print(f'\t\tIgnoring empty test class {tc.get_class_path()}')
    my_packages.append(package)
    
project: Project = Project()
project.packages = my_packages

In [172]:
for package in project.packages:
    print(package.yaml_dump())

package: anagrams
classes:
- anagrams.Anagrams
test_classes:
-   class_path: anagrams.metaAI.prompt0.AnagramsTest
    llm: metaAI
    prompt: prompt0
-   class_path: anagrams.ai21Jamba.prompt0.AnagramsTest
    llm: ai21Jamba
    prompt: prompt0
-   class_path: anagrams.chatGPT4o.prompt0.AnagramsChatGPT4o
    llm: chatGPT4o
    prompt: prompt0
-   class_path: anagrams.chatGPTo1Preview.prompt0.anagramschatGPTo1PreviewTest
    llm: chatGPTo1Preview
    prompt: prompt0
-   class_path: anagrams.claudeSonnet.prompt0.AnagramsTestClaude35Sonnet
    llm: claudeSonnet
    prompt: prompt0

package: binaryTree
classes:
- binaryTree.BinaryTree
test_classes:
-   class_path: binaryTree.metaAI.prompt0.BinaryTreeTest
    llm: metaAI
    prompt: prompt0
-   class_path: binaryTree.chatGPT4o.prompt0.binaryTreechatGPT4oTest
    llm: chatGPT4o
    prompt: prompt0
-   class_path: binaryTree.chatGPTo1Preview.prompt0.binaryTreechatGPTo1PreviewTest
    llm: chatGPTo1Preview
    prompt: prompt0
-   class_path: b

In [104]:
project.get_classes(package_filter='anagrams')

[Class(name=Anagrams, package=anagrams)]

In [106]:
project.get_test_classes(llm_filter='ai21Jamba')

[TestClass(name=AnagramsTest, package=anagrams, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=orangesRotTest, package=orangesRotting, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=NumIslandsTest, package=numIslands, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=CloneGraphTest, package=cloneGraph, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=NqueensTest, package=NQueens, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=MedianTest, package=MedianSortedArrays, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=RegularTest, package=RegularExpressionMatching, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=TritypTest, package=trityp, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=TritypTestAI21Jamba, package=trityp, llm=ai21Jamba, prompt=prompt0)]

In [122]:
import subprocess
debug=True
def run_maven_clean():
    """Clean the project to remove any existing compiled classes and any existing reports."""
    debug_print("Cleaning project...")
    result = subprocess.run(["mvn", "clean"], capture_output=True, text=True, cwd=root_path)
    if result.returncode != 0:
        print("Error cleaning project")
        print(result.stderr)
        sys.exit(1)
    debug_print("Clean successful.")

In [123]:
run_maven_clean()

DEBUG:Cleaning project...
DEBUG:Clean successful.


In [124]:
def run_maven_compile():
    """Compile all classes in the project"""
    debug_print("Compiling project...")
    result = subprocess.run(["mvn", "compile"], capture_output=True, text=True, cwd=root_path)
    if result.returncode != 0:
        print("Error during compilation:")
        print(result.stderr)
        sys.exit(1)
    debug_print("Compilation successful.")

In [125]:
run_maven_compile()

DEBUG:Compiling project...
DEBUG:Compilation successful.


In [130]:
def run_maven_test_compile():
    """Compile all test classes in the project"""
    debug_print("Compiling project test classes...")
    result = subprocess.run(["mvn", "test-compile"], capture_output=True, text=True, cwd=root_path)
    if result.returncode != 0:
        print("Error during test compilation:")
        print(result.stdout)
        print(result.stderr)
        sys.exit(1)
    debug_print("Test compilation successful.")


In [131]:
run_maven_test_compile()

DEBUG:Compiling project test classes...
DEBUG:Test compilation successful.


In [ ]:
def run_pit_for_single_test_class(target_class: Class, test_class: TestClass):
    """Run PIT mutation testing for a specific target class and test class."""
    debug_print(f"Running PIT for class '{target_class.get_class_path()}' with test '{test_class.get_class_path()}'...")
    
    # Construct the PIT command
    pit_command = [
        "mvn", "org.pitest:pitest-maven:mutationCoverage",
        f"-DtargetClasses={target_class.get_class_path()}",
        f"-DtargetTests={test_class.get_class_path()}",
        "-Dverbose=true"  # Optional: add for more detailed output
    ]

    # Execute the PIT command
    result = subprocess.run(pit_command, capture_output=True, text=True, cwd=root_path)
    if result.returncode != 0:
        print("Error during PIT execution:")
        print(result.stderr)
        sys.exit(1)
    
    # Print PIT output
    print(result.stdout)
    # TODO: parse output to ensure test was successful
    debug_print("PIT testing succeeded?")

In [153]:
def run_jacoco_for_single_test_class(test_class: TestClass):
    """Execute tests with jacoco coverage"""
    maven_command = ["mvn", "test", f"-Dtest={test_class.get_class_path()}"]
    debug_print("Running test suite with jacoco coverage for {test_class.name}...")
    try:
        result = subprocess.run(maven_command, capture_output=True, text=True, cwd=root_path)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running tests for {test_class.name}")
        print(e.stdout)
        print(e.stderr)
        return
    # TODO: Parse results to verify that the test run actually ran tests and generated a coverage report
    # Check if any tests were actually run
    if ("Tests run: 0" in result.stdout):
        print(f"Error: Test class {test_class.name} failed to run any tests.")
        print(result.stdout)
        return
    debug_print(result.stdout)
    debug_print("Test run succeeded?")

In [154]:
import xml.etree.ElementTree as ET

def parse_jacoco(jacoco_path, target_package=None, target_class=None):
    try:
        tree = ET.parse(jacoco_path)
        root = tree.getroot()

        # Find the package(s) matching the target package or class
        for package in root.findall('package'):
            package_name = package.get('name')
            if target_package and package_name != target_package:
                continue

            print(f"\nPackage: {package_name}")
            
            # Find each class in the package
            for cls in package.findall('class'):
                class_name = cls.get('name').replace('/', '.')
                if target_class and not class_name.endswith(target_class):
                    continue

                print(f"  Class: {class_name}")

                # Extract coverage metrics
                metrics = {}
                for counter in cls.findall('counter'):
                    counter_type = counter.get('type')
                    missed = int(counter.get('missed'))
                    covered = int(counter.get('covered'))
                    total = missed + covered
                    coverage = (covered / total * 100) if total > 0 else 0
                    metrics[counter_type] = coverage

                # print metrics
                if (instruction_coverage := metrics.get('INSTRUCTION')):
                    instruction_coverage = f"{instruction_coverage:.2f}%"
                else:
                    instruction_coverage = 'N/A'
                print(f"    Instruction Coverage: {instruction_coverage}")
                if (branch_coverage := metrics.get("BRANCH")):
                    branch_coverage = f"{branch_coverage:.2f}%"
                else:
                    branch_coverage = "N/A"
                print(f"    Branch Coverage: {branch_coverage}")
                if (line_coverage := metrics.get('LINE')):
                    line_coverage = f"{line_coverage:.2f}%"
                else:
                    line_coverage = "N/A"
                print(f"    Line Coverage: {line_coverage}")

    except ET.ParseError:
        print("Error: Could not parse jacoco.xml. Check if the file is valid XML.")
    except FileNotFoundError:
        print(f"Error: File {jacoco_path} not found. Ensure the file path is correct.")

In [155]:
anagrams = project.get_package('anagrams')

In [156]:
anagrams

Package(name=anagrams, classes=[Class(name=Anagrams, package=anagrams)], test_classes=[TestClass(name=AnagramsTest, package=anagrams, llm=metaAI, prompt=prompt0), TestClass(name=AnagramsTest, package=anagrams, llm=ai21Jamba, prompt=prompt0), TestClass(name=AnagramsChatGPT4o, package=anagrams, llm=chatGPT4o, prompt=prompt0), TestClass(name=anagramschatGPTo1PreviewTest, package=anagrams, llm=chatGPTo1Preview, prompt=prompt0), TestClass(name=AnagramsTestClaude35Sonnet, package=anagrams, llm=claudeSonnet, prompt=prompt0)])

In [157]:
anagrams.test_classes

[TestClass(name=AnagramsTest, package=anagrams, llm=metaAI, prompt=prompt0),
 TestClass(name=AnagramsTest, package=anagrams, llm=ai21Jamba, prompt=prompt0),
 TestClass(name=AnagramsChatGPT4o, package=anagrams, llm=chatGPT4o, prompt=prompt0),
 TestClass(name=anagramschatGPTo1PreviewTest, package=anagrams, llm=chatGPTo1Preview, prompt=prompt0),
 TestClass(name=AnagramsTestClaude35Sonnet, package=anagrams, llm=claudeSonnet, prompt=prompt0)]

In [158]:
test_classes = anagrams.test_classes
test_class = test_classes[0]
run_jacoco_for_single_test_class(test_class)

DEBUG:Running test suite with jacoco coverage for {test_class.name}...
DEBUG:[INFO] Scanning for projects...
[INFO] 
[INFO] --------------------< com.example:my-maven-project >--------------------
[INFO] Building my-maven-project 1.0-SNAPSHOT
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- jacoco:0.8.8:prepare-agent (default) @ my-maven-project ---
[INFO] argLine set to -javaagent:/home/chris/.m2/repository/org/jacoco/org.jacoco.agent/0.8.8/org.jacoco.agent-0.8.8-runtime.jar=destfile=/home/chris/Documents/UT/SWTest/project/QualitySoftware/target/jacoco.exec
[INFO] 
[INFO] --- resources:3.3.1:resources (default-resources) @ my-maven-project ---
[WARNING] Using platform encoding (UTF-8 actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] skip non existing resourceDirectory /home/chris/Documents/UT/SWTest/project/QualitySoftware/src/main/resources
[INFO] 
[INFO] --- compiler:3.8.1:compile (d

In [146]:
run_maven_test_compile()

DEBUG:Compiling project test classes...
DEBUG:Test compilation successful.


In [159]:
jacoco_path = root_path + 'target/site/jacoco/jacoco.xml'
target_package = 'anagrams'
target_class = 'Anagrams'

In [160]:
parse_jacoco(jacoco_path, target_package, target_class)


Package: anagrams
  Class: anagrams.Anagrams
    Instruction Coverage: 100.00%
    Branch Coverage: 100.00%
    Line Coverage: 100.00%


In [ ]:
Anagrams = anagrams.get_class('Anagrams')
TestAnagrams_metaAI_prompt0 = project.get_test_classes(package_filter='anagrams', llm_filter='metaAI', prompt_filter='prompt0')[0]

In [181]:
Trityp = project.get_classes(package_filter='trityp')[0]
TritypTest_ai21Jamba_prompt0 = project.get_test_classes(package_filter='trityp', llm_filter='ai21Jamba', prompt_filter='prompt0')[0]


In [184]:
TritypTest_ai21Jamba_prompt0.get_class_path()

'trityp.ai21Jamba.prompt0.TritypTest'

In [ ]:
run_maven_clean()
run_maven_compile()
run_maven_test_compile()

In [186]:
debug=True
run_pit_for_single_test_class(target_class=Anagrams, test_class=TestAnagrams_metaAI_prompt0)

DEBUG:Running PIT for class 'Anagrams' with test 'AnagramsTest'...
Error during PIT execution:
12:05:29 AM PIT >> INFO : ---------------------------------------------------------------------------
12:05:29 AM PIT >> INFO : Enabled (+) and disabled (-) features.
12:05:29 AM PIT >> INFO : -----------------------------------------
12:05:29 AM PIT >> INFO : +FANN           Filters mutations in classes and methods with matching annotations of class or runtime retention
12:05:29 AM PIT >> INFO :   [annotation]    Annotation to avoid (full package name not required)
12:05:29 AM PIT >> INFO : +FENUM          Filters mutations in enum constructors
12:05:29 AM PIT >> INFO : +FFBLOCK        Filters mutations in code duplicated by finally block inlining
12:05:29 AM PIT >> INFO : +FFEACH         Filters mutations in compiler generated code that implements for each loops
12:05:29 AM PIT >> INFO : +FFLOOP         Filters any mutations to increments in for loops as they may cause timeouts
12:05:29 AM 

NameError: name 'sys' is not defined